In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
import syft as sy  # Import Syft for Federated Learning

In [2]:
# Define the neural network model (simple example)
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = x.view(-1, 28 * 28)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [5]:
!pip uninstall syft



^C


In [6]:
!pip list


Package                                  Version
---------------------------------------- -----------
absl-py                                  2.1.0
aiohttp                                  3.9.5
aiosignal                                1.3.1
annotated-types                          0.7.0
anyio                                    4.4.0
argon2-cffi                              23.1.0
argon2-cffi-bindings                     21.2.0
asgiref                                  3.8.1
asttokens                                2.4.1
astunparse                               1.6.3
asyncache                                0.3.1
attrs                                    23.2.0
azure-core                               1.30.2
azure-storage-blob                       12.19.1
backoff                                  2.2.1
bcrypt                                   4.1.2
blinker                                  1.8.2
boto3                                    1.34.56
botocore                                 1.3

In [13]:

# Create a virtual Machine for the server
server = sy.Worker( name="server")

# Create virtual Machines for the clients
client1 = sy.Worker( name="client1")
client2 = sy.Worker( name="client2")

In [14]:
# Prepare the training data (for demonstration purposes, use the same data for all clients)
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
train_dataset = datasets.MNIST(root="./data", train=True, transform=transform, download=True)

# Split the data between the clients
client1_data, client2_data = torch.utils.data.random_split(train_dataset, [30000, 30000])

In [23]:


# # Send data to clients
# client1_dataset = sy.BaseDataset(client1_data, transform=transform).send(client1)
# client2_dataset = sy.BaseDataset(client2_data, transform=transform).send(client2)

# Load data into DataLoader
client1_loader = torch.utils.data.DataLoader(client1_data, batch_size=32, shuffle=True)
client2_loader = torch.utils.data.DataLoader(client2_data, batch_size=32, shuffle=True)

# Initialize the global model
global_model = SimpleNN()

In [29]:
# Training function for each client
def train(model, data_loader, optimizer, criterion, device):
    model.train()
    model
    for batch_idx, (data, target) in enumerate(data_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
    model.get()  # Get the model back to the server
    return model

In [31]:
# Federated learning loop
for epoch in range(5):  # Number of global epochs
    print(f"Epoch {epoch + 1}")
    


    # Distribute model to clients
    client_models = [global_model for client in [client1, client2]]
    # Load the initial state of the global model into each client model
    for model in client_models:
        model.load_state_dict(global_model.state_dict())
    optimizers = [optim.SGD(model.parameters(), lr=0.01) for model in client_models]
    criterion = nn.CrossEntropyLoss()

    # Train the model on each client
    for client_model, client_loader, client_optimizer, client in zip(client_models, [client1_loader, client2_loader], optimizers, [client1, client2]):
        client_model = train(client_model, client_loader, client_optimizer, criterion, client)

    # Average the weights
    with torch.no_grad():
        global_state_dict = global_model.state_dict()
        for key in global_state_dict:
            global_state_dict[key] = torch.stack([client_model.state_dict()[key].float() for client_model in client_models]).mean(0)

    global_model.load_state_dict(global_state_dict)
    print("Updated global model")

print("Training complete.")

Epoch 1


TypeError: to() received an invalid combination of arguments - got (Worker), but expected one of:
 * (torch.device device = None, torch.dtype dtype = None, bool non_blocking = False, bool copy = False, *, torch.memory_format memory_format = None)
 * (torch.dtype dtype, bool non_blocking = False, bool copy = False, *, torch.memory_format memory_format = None)
 * (Tensor tensor, bool non_blocking = False, bool copy = False, *, torch.memory_format memory_format = None)
